# Procesamiento de fuentes de datos

Aquí documentamos los procedimientos con los que procesamos algunas fuentes de datos que se usan para el análisis de uso de suelo y que no son parte de la librería.

In [ ]:
import pandas as pd
import geopandas as gpd
from pyLandUseMX.descargas import *
from pathlib import Path
import os

## Censo a nivel manzana

Aquí documentamos el procedimiento para obtener los datos a nivel manzana que se utilizan en `Coberturas.agrega_mananas()`

### Datos de ejemplo CDMX

Esta base se usa como ejemplo en la librería.

In [ ]:
# Descargamos la geometría de las manzanas
Path(DOWNLOADS_PATH).mkdir(parents=True, exist_ok=True)
fname = 'manzanas.gpkg'
absp = os.path.abspath(os.path.join(DOWNLOADS_PATH, fname))
if os.path.exists(absp):
    print("El archivo ya está descargado")
else:
    url = 'https://www.dropbox.com/s/bzsj7wl4izzdbjh/manzanas_identificadores.gpkg?dl=1'
    r = requests.get(url, allow_redirects=True)
    open(absp, 'wb').write(r.content)
# Leemos la geometría de las manzanas de CDMX
manzanas = gpd.read_file(absp)
# Leemos los datos del censo (sólo unas columnas para elejemplo)
censo = pd.read_csv("../../datos/censo_manzanas.zip", 
                    dtype={'CVEGEO':str, 'colonia_cve': 'Int64'}, 
                    usecols=['CVEGEO', 'OCUPVIVPAR', 'VIVPAR_HAB'])

# Unimos 
manzanas = manzanas.merge(censo, on='CVEGEO').fillna(0) # Llenamos con 0 los faltantes
manzanas = manzanas.drop(columns=['colonia_cve', 'cuadrante_id', 'colonia_nombre', 'AMBITO', 'TIPOMZA'])
manzanas.head()

El archivo ya está descargado


,CVEGEO,geometry,VIVPAR_HAB,OCUPVIVPAR
0,0901000010898031,"POLYGON ((2790204.135 820779.247, 2790125.593 ...",22.0,93.0
1,0901000012269024,"POLYGON ((2791387.916 821465.156, 2791376.215 ...",0.0,0.0
2,0901000011472068,"POLYGON ((2788494.355 822775.106, 2788478.422 ...",33.0,124.0
3,0901000011824024,"POLYGON ((2790870.494 819456.298, 2790862.256 ...",95.0,340.0
4,0901000012377004,"POLYGON ((2792111.138 821977.886, 2792115.437 ...",23.0,82.0


Lo guardamos

In [ ]:
manzanas.to_file("../../datos/descargas/manzanas_ejemplo.shp")